In [14]:
from sklearn.model_selection import train_test_split

# Data Processesing
This notebook segment outlines the initial steps of preparing a dataset for training a machine learning model, specifically focusing on location-based data. The code is divided into two main parts: data import and preparation of training and test data.

## Data Import
- The dataset is imported from a CSV file named `training_data.csv` using Pandas.
- Any rows with missing values are dropped to ensure data quality and consistency. This is crucial for models that are sensitive to null values.
- A specific preprocessing step is included to address an identified issue: rows where the 'org_location' field is 0.0 are removed. This step is noted as necessary due to a peculiar behavior of some models treating 0.0 as a null value.

## Preparing Training and Test Data
- The dataset is split into features (`x`) and the target variable (`y`), which is 'org_location' in this case.
- The data is then divided into training and test sets using the `train_test_split` method from Scikit-Learn. 20% of the data is reserved for testing, and the split is performed with shuffling to ensure randomization, using a random state of 2 for reproducibility.
- Further, the training set is split again to create a validation set, also using a 20% split with shuffling and the same random state. This additional split is important for model validation during training.
- The training, test, and validation sets are saved into separate CSV files for easy access in later stages of the modeling process.

The print statements at the end of each major section provide a quick summary of the dataset sizes, offering a clear understanding of how much data is available for training, validation, and testing.

# Changelog
**V0.1**: Compleated the main operartion of the models including printing results to df. 
**V0.2**: Overfitting is a problem with many of the models. For others, the problem is very poor accruacy and memory overhead - changed the number of training samples, introduced SMOTE for class imbalance, Feature scaling and hyperparameter tunning with Gridsearch among other changes 
**V0.3**: Changed the git directory and cleaned this - research organsiation changes mostly
**V0.4**: Changed the number of training samples to 10 (down from 100), removed the k-seed string (opting for runtime endoded and occruance counts only)
**V0.5**: included SHAP in models to explore feature importance 

In [15]:
""" This cell generates the fundamental index dataset within 'reference genomes' for the sample set. """

import os
import sys
import pandas as pd
from Bio.Seq import Seq
from collections import OrderedDict

sys.path.append("C:/Users/Tony/OneDrive - Ulster University (1)/Phd - FPGA acceleration in genomics/Publications + Writing/PhD_reports/PhD Thesis/cp3. Seed generation via ML/research")

from research.PyPair.io_processing import IO_processing
from research.PyPair.gen_index import GenIndex

config = {
    "seed_length": 28,
    "read_length": 100,
    "training_iterations": 8,
}

""" Experiment specific functions """
def reverse_complement(sequence_set):
    """ Compute reverse compliment of k-seed string """
    sequence_strings = sequence_set['k-seed']

    rc_strings = []
    clean_rc_strings = []
    for string in sequence_strings:
        s_seq = Seq(string)
        rc_string = s_seq.reverse_complement()
        rc_strings.append(rc_string)

    for row in rc_strings:
        new_string = ''.join(row)
        clean_rc_strings.append(new_string)

    sequence_set['rc_seeds'] = clean_rc_strings
    return sequence_set

def encode(_index):
    """encode the data set using 2bit encoding"""
    encoded = []
    dictionary = {'$': '', ',':'', 'A': '00', 'C': '01', 'G': '10', 'T': '11'}

    if isinstance(_index, str):
        transTable = _index.maketrans(dictionary)
        txt = _index.translate(transTable)
        encoded = txt
    else:
        for row in _index:
            encoded_row = row.translate(row.maketrans(dictionary))
            encoded.append(encoded_row)
    return encoded

def extend_dataset(_df, _config):
    """ Extend training set with repeated instances of class example """
    classes = _df['org_location'].unique()
    extended_df = pd.DataFrame()

    for cls in classes:
        cls_df = _df[_df['org_location'] == cls]
        repeat_times = max(1, _config['training_iterations'] - cls_df.shape[0])

        extended_cls_df = pd.concat([cls_df] * repeat_times, ignore_index=True)
        extended_df = pd.concat([extended_df, extended_cls_df], ignore_index=True)

    return extended_df

""" Data import and standard preprocessing functions """
def process_fasta_file(_file_path, _config):
    """ Parse fasta file """
    io_processor = IO_processing()
    _sequence_id, _sequence_string, _sequence_length = io_processor.pharse_reference(_file_path, _config['seed_length'])
    return _sequence_id, _sequence_string, _sequence_length

def index_sequence(_sequence_string, seed_length):
    """ Index sequence using GenIndex() """
    index_obj = GenIndex()
    _index = index_obj.generate_index(_sequence_string, seed_length)
    return _index

def save_index_as_csv(_index, file_name):
    """ Save index function """
    _index_df = pd.DataFrame(_index)
    if os.path.exists(file_name):
        os.remove(file_name)
    _index_df.to_csv(file_name, index=False)

def encode_dataframe(_df):
    """ Encode dataframe """
    encoded_df = _df.copy()
    for column in encoded_df.columns:
        if column != 'org_location':
            encoded_df[column] = encoded_df[column].apply(lambda _x: encode(_x) if isinstance(_x, str) else _x)
    return encoded_df

def runLengthEncoding(_input):
    """ Run length encoding of k-seed string """
    _dict = OrderedDict.fromkeys(_input, 0)

    for ch in _input:
        _dict[ch] += 1
    _output = ''
    characters = ''
    _values = ''
    for key, _value in _dict.items():
        _output = _output + key + str(_value)
        characters = characters + key
        _values = _values + str(_value)
    return _output, characters, _values


""" 
remove all BWT aspects including occ array. Use only k-seed in mapping with rev compliment. If memory is a prob try with run length encoded params. 
"""

folder_path = 'C:/Users/Tony/OneDrive - Ulster University (1)/Phd - FPGA acceleration in genomics/Publications + Writing/PhD_reports/PhD Thesis/cp3. Seed generation via ML/research/PyPair/reference_samples'
fasta_files = [f for f in os.listdir(folder_path) if f.endswith('.fasta')]

for i, fasta_file in enumerate(fasta_files, start=1):
    file_path = os.path.join(folder_path, fasta_file)

    sequence_id, sequence_string, sequence_length = process_fasta_file(file_path, config)
    index = index_sequence(sequence_string, config['seed_length'])
    index_df = pd.DataFrame(index)

    reverse_complement_df = reverse_complement(index_df) # reverse compliment 

    # run length encoding 
    output = []
    character = []
    value = []

    for seed_string in reverse_complement_df['k-seed']:
        out, char, val = runLengthEncoding(seed_string)
        output.append(out)
        character.append(char)
        value.append(val)

    reverse_complement_df['runlength_output'] = output
    reverse_complement_df['runlength_character'] = character
    reverse_complement_df['runlength_value'] = value

    del output
    del character
    del value

    rc_output = []
    rc_character = []
    rc_value = []

    for rc_seed_string in reverse_complement_df['rc_seeds']:
        rc_out, rc_char, rc_val = runLengthEncoding(rc_seed_string)
        rc_output.append(rc_out)
        rc_character.append(rc_char)
        rc_value.append(rc_val)

    reverse_complement_df['runlength_rc_output'] = rc_output
    reverse_complement_df['runlength_rc_character'] = rc_character
    reverse_complement_df['runlength_rc_value'] = rc_value

    del rc_output
    del rc_character
    del rc_value

    # encode dataset 
    encoded_index = encode_dataframe(reverse_complement_df) #encodes strings 

    # filter columns based on training data analysis PCA 
    df_training_data = pd.DataFrame({
        "first":encoded_index['first'],
        "A":encoded_index['A'],
        "C":encoded_index['C'],
        "G":encoded_index['G'],
        "T":encoded_index['T'],
        # "k-seed":encoded_index['k-seed'],
        "rc_seeds":encoded_index['rc_seeds'],
        "runlength_output":encoded_index['runlength_output'],
        "runlength_value":encoded_index['runlength_value'],
        "runlength_rc_output":encoded_index['runlength_rc_output'],
        "runlength_rc_character":encoded_index['runlength_rc_character'],
        "runlength_rc_value":encoded_index['runlength_rc_value'],
        "org_location":encoded_index['org_location'],
    })

    # Extend the dataset
    extended_encoded_index = extend_dataset(df_training_data, config)

    # Save the extended encoded index
    extended_output_file_name = os.path.join(folder_path, f'S{i}_extended_encoded_reference.csv')
    save_index_as_csv(extended_encoded_index, extended_output_file_name)


... Pharsing input reference genome

pharsing reference sequence C:/Users/Tony/OneDrive - Ulster University (1)/Phd - FPGA acceleration in genomics/Publications + Writing/PhD_reports/PhD Thesis/cp3. Seed generation via ML/research/PyPair/reference_samples\s1.fasta
sequence ID: CM001012.3
sequence Length: 10000
Making rotations
Building suffix array column
Building BWT
Populating FM index character counts
Generating k-length seeds
Cannot find reference_genomes directory, creating one...


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data_sets'

In [ ]:
folder_path = 'reference_samples'
reference_files = [f for f in os.listdir(folder_path) if f.endswith('extended_encoded_reference.csv')]

for ref_file in reference_files:
    # Data import
    file_path = os.path.join(folder_path, ref_file)
    df = pd.read_csv(file_path)
    df = df.dropna()
    df = df[df['org_location'] != 0.0]  # Handle the error with models treating 0.0 as null
    #df = df.drop(columns=['suffix_array', 'k-seed-extend'])

    # Preparing training and test data
    x = df.loc[:, df.columns != 'org_location']
    y = df.loc[:, 'org_location']
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=True, random_state=42)

    # Saving the datasets in the same folder
    base_filename = ref_file.split('_reference.csv')[0]
    X_train.to_csv(os.path.join(folder_path, f"{base_filename}_x_train.csv"), index=False)
    X_test.to_csv(os.path.join(folder_path, f"{base_filename}_X_test.csv"), index=False)
    y_train.to_csv(os.path.join(folder_path, f"{base_filename}_y_train.csv"), index=False)
    y_test.to_csv(os.path.join(folder_path, f"{base_filename}_y_test.csv"), index=False)
    X_val.to_csv(os.path.join(folder_path, f"{base_filename}_X_val.csv"), index=False)
    y_val.to_csv(os.path.join(folder_path, f"{base_filename}_y_val.csv"), index=False)

    print(f"Processed {ref_file}:")
    print(f"  X training dataset length: {len(X_train)}")
    print(f"  y training dataset length: {len(y_train)}")


# Model Training
This section of the notebook focuses on the training of multiple machine learning models using Scikit-Learn version 0.24.2. The models selected represent different archetypes in machine learning, including Nearest Neighbour, Averaging Methods, Naïve Bayes, and Gradient Boosting. The specific models and their roles are as follows:

## Models Overview

1. **Nearest Centroid (Nearest Neighbour Archetype)**: 
   - This model is a simplistic yet effective approach for classification, based on the concept of the nearest centroid. It can be particularly useful for baseline comparisons.

2. **KNeighbours Classifier (Nearest Neighbour Archetype)**: 
   - A versatile and widely-used model that classifies data based on the closest training examples in the feature space. It's effective for datasets where the decision boundary is irregular.

3. **Random Forest (Averaging Methods Archetype)**: 
   - A robust ensemble learning method, Random Forest constructs a multitude of decision trees during training and outputs the class that is the mode of the classes (classification) of the individual trees.

4. **Extra Trees Classifier (Averaging Methods Archetype)**: 
   - Similar to Random Forest, this model fits a number of randomized decision trees on various sub-samples of the dataset and uses averaging to improve predictive accuracy and control over-fitting.

5. **Decision Tree (Averaging Methods Archetype)**: 
   - A decision tree is a flowchart-like structure where an internal node represents a feature(or attribute), the branch represents a decision rule, and each leaf node represents the outcome.

6. **Gaussian Naïve Bayes (Naïve Bayes Archetype)**: 
   - This model applies the Bayes theorem with the “naive” assumption of independence between every pair of features. Gaussian Naïve Bayes is particularly suited when the features have continuous values.

7. **LGBM Classifier (Gradient Boosting Archetype)**: 
   - Light Gradient Boosting Machine is a gradient boosting framework that uses tree-based learning algorithms. It's designed for distributed and efficient training, particularly on large datasets.

8. **XGB Classifier (Gradient Boosting Archetype)**: 
   - XGBoost (Extreme Gradient Boosting) is an optimized distributed gradient boosting library. It is highly efficient, flexible, and portable, often delivering state-of-the-art performance in many machine learning tasks.

## Training Process
Each model is trained on the prepared dataset, and the performance is evaluated using the validation set. The training involves tuning model parameters to find the optimal configuration for each model. Key metrics like accuracy, precision, recall, and F1 score are used to assess each model's performance. The diversity in the selected models ensures a comprehensive examination of the dataset, as each model type brings its strengths and weaknesses to different types of data. This approach allows for a thorough understanding of which models are best suited for the specific characteristics of the dataset in question.The results from these models can be used to benchmark performance and guide the selection of the most suitable model for deployment.

 ## Nearest Neighbour
Nearest Neighbor models are a fundamental class of algorithms in the field of machine learning, predominantly used for classification tasks, though they can also be employed for regression. These models operate on the principle of similarity, identifying the closest data points in the training set to make predictions for new, unseen data. The simplicity, intuitiveness, and effectiveness of these models make them an essential part of any data scientist's toolkit.

#### Key Concepts

1. **Basic Principle**: The core idea behind nearest neighbor models is that similar data points are close to each other in the feature space. Therefore, the label or value of a new data point can be predicted based on the labels or values of its nearest neighbors in the training set.

2. **Distance Metrics**: These models rely on distance metrics to determine the closeness of data points. Common metrics include Euclidean, Manhattan, and Hamming distances. The choice of metric can significantly impact the model's performance and is often dictated by the nature of the data.

3. **K-Nearest Neighbors (KNN)**: One of the most popular variants is the K-Nearest Neighbors algorithm. KNN uses a predefined number 'K' to determine the number of neighboring data points to consider for making a prediction. The optimal value of 'K' is typically selected through cross-validation.

4. **Nearest Centroid Classifier**: This variant classifies data points based on the closest centroid of the training samples in the feature space. It's particularly useful when the data is well-separated into clusters.

5. **Applications**: Nearest neighbor models are used in a wide range of applications, including image recognition, recommendation systems, and medical diagnosis, where the assumption that similar instances have similar outcomes holds true.

6. **Strengths and Limitations**:
   - **Strengths**: These models are easy to implement, interpret, and don't require assumptions about the underlying data distribution. They're also highly adaptable to changes in the input data.
   - **Limitations**: Nearest neighbor models can suffer from high computational costs, especially with large datasets, and can be sensitive to irrelevant or redundant features.

#### Practical Implementation

In practical scenarios, implementing nearest neighbor models involves careful preprocessing of data, selection of an appropriate distance metric, and tuning of parameters like 'K' in KNN. It's also crucial to scale or normalize the data, as these models are sensitive to the scale of the input features.

Moreover, modern applications might require optimizations for handling large datasets, such as using approximate nearest neighbor search algorithms or efficient data structures like KD-trees and Ball Trees.

In summary, nearest neighbor models are a versatile and straightforward tool for both classification and regression tasks in machine learning. Their ability to adapt to complex, real-world datasets makes them a valuable component of the machine learning workflow.


In [ ]:
import pandas as pd 
S1_X_train = pd.read_csv('reference_samples/S1_extended_encoded_X_train.csv')
S1_X_test = pd.read_csv('reference_samples/S1_extended_encoded_X_test.csv')
S1_y_train = pd.read_csv('reference_samples/S1_extended_encoded_y_train.csv')
S1_y_test = pd.read_csv('reference_samples/S1_extended_encoded_y_test.csv')

S2_X_train = pd.read_csv('reference_samples/S2_extended_encoded_X_train.csv')
S2_X_test = pd.read_csv('reference_samples/S2_extended_encoded_X_test.csv')
S2_y_train = pd.read_csv('reference_samples/S2_extended_encoded_y_train.csv')
S2_y_test = pd.read_csv('reference_samples/S2_extended_encoded_y_test.csv')

S3_X_train = pd.read_csv('reference_samples/S3_extended_encoded_X_train.csv')
S3_X_test = pd.read_csv('reference_samples/S3_extended_encoded_X_test.csv')
S3_y_train = pd.read_csv('reference_samples/S3_extended_encoded_y_train.csv')
S3_y_test = pd.read_csv('reference_samples/S3_extended_encoded_y_test.csv')

S4_X_train = pd.read_csv('reference_samples/S4_extended_encoded_X_train.csv')
S4_X_test = pd.read_csv('reference_samples/S4_extended_encoded_X_test.csv')
S4_y_train = pd.read_csv('reference_samples/S4_extended_encoded_y_train.csv')
S4_y_test = pd.read_csv('reference_samples/S4_extended_encoded_y_test.csv')


### Nearest Centroid
Nearest Centroid model classifies data by assigning each observation to the class of the nearest centroid, simplifying computation and making it particularly effective for large, well-separated datasets.

- **Ensuring Numeric Data**:
  - **Converting Data to Numeric**: We use `pd.to_numeric` to convert all feature columns to numeric types, coercing non-numeric values to NaN. This helps standardize the data and ensure consistency.
  - **Reference**: [Pandas to_numeric Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html)

- **Handling NaN Values**:
  - **Forward-Filling and Dropping NaNs**: NaN values are filled using the forward-fill method (`fillna(method='ffill')`), and any remaining NaNs are dropped using `dropna()`. This helps in cleaning the data and avoiding issues during model training.
  - **Reference**: [Pandas fillna Documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

- **Handling Infinite Values**:
  - **Replacing Infinite Values with NaNs**: Infinite values are replaced with NaNs (`replace([np.inf, -np.inf], np.nan)`), and these NaNs are subsequently handled using forward-filling and dropping. This ensures that the dataset does not contain values that could cause computational issues.
  - **Reference**: [Numpy isinf Documentation](https://numpy.org/doc/stable/reference/generated/numpy.isinf.html)

- **Feature Scaling**:
  - **StandardScaler**: We use `StandardScaler` to normalize the features to have a mean of 0 and a standard deviation of 1. This helps in improving the performance of many machine learning algorithms.
  - **Reference**: [Sklearn StandardScaler Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

- **Handling Class Imbalance**:
  - **SMOTE (Synthetic Minority Over-sampling Technique)**: SMOTE is used to oversample the minority class by generating synthetic samples. This helps in addressing class imbalance and improving the model's ability to generalize.
  - **Reference**: [Imbalanced-learn SMOTE Documentation](https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html)

- **Hyperparameter Tuning**:
  - **GridSearchCV**: We use `GridSearchCV` to perform an exhaustive search over specified hyperparameter values. This helps in finding the best model parameters and improving model performance.
  - **Reference**: [Sklearn GridSearchCV Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

- **Cross-Validation**:
  - **StratifiedShuffleSplit**: This cross-validation strategy ensures that each fold of the cross-validation split preserves the percentage of samples for each class, providing a robust evaluation of model performance.
  - **Reference**: [Sklearn StratifiedShuffleSplit Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html)

These methods collectively help in cleaning the data, handling class imbalance, tuning model parameters, and ultimately addressing overfitting, leading to a more robust and generalizable model.


In [ ]:
import pandas as pd
import numpy as np
import time
import joblib
import warnings
import shap
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import NotFittedError
from memory_profiler import memory_usage

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

def profile_memory(_model, _X_test):
    try:
        _mem_usage = memory_usage((_model.predict, (_X_test,)), interval=0.1)
        mem_usage = max(_mem_usage) - min(_mem_usage)
        if mem_usage == 0:
            mem_usage = np.nan
    except NotFittedError:
        mem_usage = np.nan
    return mem_usage

for X_train, X_test, y_train, y_test in datasets:
    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Ensure all data is numeric
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')
    y_train = pd.Series(y_train).apply(pd.to_numeric, errors='coerce')
    y_test = pd.Series(y_test).apply(pd.to_numeric, errors='coerce')

    # Handle NaN values
    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    y_train = pd.Series(y_train).ffill().dropna().values
    y_test = pd.Series(y_test).ffill().dropna().values

    # Handle infinite values
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_train = np.where(np.isinf(y_train), np.nan, y_train)
    y_test = np.where(np.isinf(y_test), np.nan, y_test)

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    y_train = pd.Series(y_train).ffill().dropna().values
    y_test = pd.Series(y_test).ffill().dropna().values

    # Ensure data consistency
    feature_names = X_train.columns.tolist()  # Save column names for SHAP
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train
    y_test = y_test

    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train_scaled = np.vstack([X_train_scaled, X_train_scaled[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train_scaled, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'metric': ['manhattan', 'euclidean'],
        'shrink_threshold': [None, 0.1, 0.5, 1.0]
    }

    model = NearestCentroid()

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Measure memory usage during prediction
    mem_usage = profile_memory(best_model, X_test_scaled)

    # Measure execution time during prediction
    start_time = time.time()
    predicted = best_model.predict(X_test_scaled)
    execution_time = time.time() - start_time

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')

    execution_time_per_read = execution_time / len(X_test_scaled)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test_scaled) if not np.isnan(mem_usage) else np.nan  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save model weights (centroids)
filename = 'model_weights/centroid_model.joblib'

if hasattr(best_model, 'centroids_'):
    model_weights = best_model.centroids_
    joblib.dump(model_weights, filename)
else:
    print("The model does not have the attribute 'centroids_'")
    joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.KernelExplainer(best_model.predict, X_train_resampled[:100])  # Use a subset of the training data
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test

In [ ]:
NearestCentroid_shap_values = pd.DataFrame(shap_values)
NearestCentroid_shap_values['Classifier'] = 'NearestCentroid'
NearestCentroid_shap_values

In [ ]:
NearestCentroid_results = pd.DataFrame(aggregate_metrics)
NearestCentroid_results['Model'] = 'Nearest Centroid'
NearestCentroid_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
NearestCentroid_results = NearestCentroid_results[['Model',
                                   'Sample',
                                   'Mean CV F1 Score',
                                   'Precision',
                                   'Recall',
                                   'F1 Score',
                                   'Memory (MB) per read mapping',
                                   'Execution Time (s) per read mapping',
                                   'Memory (MB) 50k batch',
                                   'Execution Time (s) 50k batch']]
NearestCentroid_results

### K-Nearest Neighbour
The Nearest Neighbor algorithm is a simple and intuitive classification method that predicts the label of a new data point based on the most common label among its closest neighbors in the feature space.

In [ ]:
import pandas as pd
import numpy as np
import time
import joblib
import warnings
import shap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from memory_profiler import memory_usage
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import NotFittedError

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

def profile_memory(_model, _X_test):
    _mem_usage = memory_usage((_model.predict, (_X_test,)), interval=0.1)
    return max(_mem_usage) - min(_mem_usage)

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Ensure all data is numeric
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')
    y_train = pd.Series(y_train).apply(pd.to_numeric, errors='coerce')
    y_test = pd.Series(y_test).apply(pd.to_numeric, errors='coerce')

    # Handle NaN values
    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    y_train = pd.Series(y_train).ffill().dropna().values
    y_test = pd.Series(y_test).ffill().dropna().values

    # Handle infinite values
    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
    y_train = np.where(np.isinf(y_train), np.nan, y_train)
    y_test = np.where(np.isinf(y_test), np.nan, y_test)

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    y_train = pd.Series(y_train).ffill().dropna().values
    y_test = pd.Series(y_test).ffill().dropna().values

    # Ensure data consistency
    feature_names = X_train.columns.tolist()  # Save column names for SHAP
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train
    y_test = y_test

    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train_scaled = np.vstack([X_train_scaled, X_train_scaled[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train_scaled, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'n_neighbors': [3, 5, 7],
        'metric': ['minkowski', 'euclidean', 'manhattan'],
        'weights': ['uniform', 'distance']
    }

    model = KNeighborsClassifier()

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Measure memory usage during prediction
    try:
        mem_usage = profile_memory(best_model, X_test_scaled)
        if mem_usage == 0:
            mem_usage = np.nan  # Ensure no zero values
    except NotFittedError:
        mem_usage = np.nan

    # Measure execution time during prediction
    start_time = time.time()
    predicted = best_model.predict(X_test_scaled)
    execution_time = time.time() - start_time

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')

    execution_time_per_read = execution_time / len(X_test_scaled)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test_scaled) if not np.isnan(mem_usage) else np.nan  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained KNN model
filename = 'model_weights/knn_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.KernelExplainer(best_model.predict, X_train_resampled[:100])  # Use a subset of the training data
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test

In [ ]:
KNeighborsClassifier_shap_values = pd.DataFrame(shap_values)
KNeighborsClassifier_shap_values['Classifier'] = 'KNeighborsClassifier'
KNeighborsClassifier_shap_values

In [ ]:
KNeighborsClassifier_results = pd.DataFrame(aggregate_metrics)
KNeighborsClassifier_results['Model'] = 'KNeighborsClassifier'
KNeighborsClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
KNeighborsClassifier_results = KNeighborsClassifier_results[['Model',
                                                   'Sample',
                                                   'Mean CV F1 Score',
                                                   'Precision',
                                                   'Recall',
                                                   'F1 Score',
                                                   'Memory (MB) per read mapping',
                                                   'Execution Time (s) per read mapping',
                                                   'Memory (MB) 50k batch',
                                                   'Execution Time (s) 50k batch']]
KNeighborsClassifier_results

### Random Forest

Random Forest is an ensemble learning method that constructs a multitude of decision trees during training and outputs the mode of the classes (for classification) or mean prediction (for regression) of the individual trees, thereby enhancing predictive accuracy and robustness against overfitting.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import joblib
import time
import gc
import warnings
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Replace NaNs and infinities
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    X_train = X_train.values
    X_test = X_test.values

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=1000):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Ensure data is in float32 format
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    model = RandomForestClassifier(criterion='gini')

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=1000)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained RandomForest model
filename = 'model_weights/random_forest_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()

In [ ]:
RandomForestClassifier_shap_values = pd.DataFrame(shap_values)
RandomForestClassifier_shap_values['Classifier'] = 'RandomForestClassifier'
RandomForestClassifier_shap_values

In [ ]:
RandomForestClassifier_results = pd.DataFrame(aggregate_metrics)
RandomForestClassifier_results['Model'] = 'RandomForestClassifier'
RandomForestClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
RandomForestClassifier_results = RandomForestClassifier_results[['Model',
                                                             'Sample',
                                                             'Mean CV F1 Score',
                                                             'Precision',
                                                             'Recall',
                                                             'F1 Score',
                                                             'Memory (MB) per read mapping',
                                                             'Execution Time (s) per read mapping',
                                                             'Memory (MB) 50k batch',
                                                             'Execution Time (s) 50k batch']]
RandomForestClassifier_results

In [ ]:
''' Aggregating NearestNeighbour results '''
NearestNeighbour_results = pd.concat([NearestCentroid_results, 
                                      KNeighborsClassifier_results, 
                                      RandomForestClassifier_results])
NearestNeighbour_shap_values = pd.concat([NearestCentroid_shap_values,
                                          KNeighborsClassifier_shap_values,
                                          RandomForestClassifier_shap_values])

## Averaging Methods
Averaging Methods, including algorithms like Random Forest and Ensemble Methods, are a critical class of machine learning models known for their robustness and accuracy. These models work by combining the predictions from multiple models to improve the overall performance, especially in terms of variance reduction.

#### Key Concepts

1. **Basic Principle**: Averaging methods involve training multiple models and combining their predictions. The final output is typically the mean (for regression tasks) or mode (for classification tasks) of the predictions from all models. This approach helps in mitigating the effects of overfitting and improving the model's generalization capabilities.

2. **Random Forest**: A popular example of averaging methods, Random Forest builds numerous decision trees and merges their outcomes. It is a form of 'bagging' where each tree is trained on a subset of the data and features, offering a diversified model performance.

3. **Ensemble Learning**: Averaging methods are a subset of ensemble learning, where the objective is to combine the strengths of various models to achieve better accuracy and stability. This includes methods like bagging and boosting.

4. **Boosting**: Another form of ensemble learning where models are trained sequentially with each model learning from the errors of its predecessors, thus focusing more on the challenging parts of the dataset.

5. **Applications**: These methods are extremely versatile and have been successfully applied in various domains, such as finance for risk assessment, healthcare for disease prediction, and natural language processing tasks.

6. **Strengths and Limitations**:
   - **Strengths**: Averaging methods are known for their high accuracy, ability to handle large datasets and feature spaces, and robustness against overfitting. They also work well with non-linear data.
   - **Limitations**: These models can be complex, computationally intensive, and less interpretable compared to simpler models like linear regression or decision trees.

#### Practical Implementation

Implementing averaging methods requires selecting the right base models and determining how to combine their predictions effectively. The choice of base models and the method of combination (like voting or averaging) can significantly impact the performance. It is also crucial to ensure diversity among the base models to maximize the benefits of averaging.

Additionally, hyperparameter tuning plays a significant role in optimizing these models. Techniques like cross-validation are essential for determining the optimal settings for parameters like the number of trees in a Random Forest or the learning rate in boosting algorithms.

In conclusion, averaging methods are a powerful set of tools in the machine learning arsenal, offering enhanced predictive performance and robustness. Their ability to combine multiple models' strengths makes them suitable for a wide range of complex real-world problems.


### Extra Trees classifier

The Extra Trees Classifier is an ensemble machine learning algorithm that operates similarly to a Random Forest but with randomization at the level of individual tree splits, offering increased variance reduction and potentially faster training at the cost of slightly higher bias.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import joblib
import time
import gc
import warnings
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Replace NaNs, infinities, and too large values
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
    X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

    X_train.ffill(inplace=True)
    X_train.dropna(inplace=True)
    X_test.ffill(inplace=True)
    X_test.dropna(inplace=True)

    X_train = X_train.values
    X_test = X_test.values

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=500):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Ensure data is in float32 format
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    model = ExtraTreesClassifier()

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=500)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained Extra Trees model
filename = 'model_weights/extra_trees_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()

In [ ]:
ExtraTreesClassifier_shap_values = pd.DataFrame(shap_values)
ExtraTreesClassifier_shap_values['Classifier'] = 'ExtraTreesClassifier'
ExtraTreesClassifier_shap_values

In [ ]:
ExtraTreesClassifier_results = pd.DataFrame(aggregate_metrics)
ExtraTreesClassifier_results['Model'] = 'ExtraTreesClassifier'
ExtraTreesClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
ExtraTreesClassifier_results = ExtraTreesClassifier_results[['Model',
                                                                 'Sample',
                                                                 'Mean CV F1 Score',
                                                                 'Precision',
                                                                 'Recall',
                                                                 'F1 Score',
                                                                 'Memory (MB) per read mapping',
                                                                 'Execution Time (s) per read mapping',
                                                                 'Memory (MB) 50k batch',
                                                                 'Execution Time (s) 50k batch']]
ExtraTreesClassifier_results

### Decision trees classifier
The Decision Trees Classifier is a versatile and interpretable machine learning algorithm that classifies data by splitting it based on feature values, creating a tree-like model of decisions and their possible consequences.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import joblib
import time
import gc
import warnings
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Convert all values to numerical types, replacing non-numerical values with NaN
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    # Replace NaNs, infinities, and too large values
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=500):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Accuracy': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Ensure data is in float32 format
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'criterion': ['gini', 'entropy']
    }

    model = DecisionTreeClassifier(random_state=0)

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=500)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')
    accuracy = accuracy_score(y_test, predicted)

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Accuracy'].append(accuracy)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained Decision Tree model
filename = 'model_weights/decision_tree_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()

In [ ]:
DecisionTreeClassifier_shap_values = pd.DataFrame(shap_values)
DecisionTreeClassifier_shap_values['Classifier'] = 'DecisionTreeClassifier'
DecisionTreeClassifier_shap_values

In [ ]:
DecisionTreeClassifier_results = pd.DataFrame(aggregate_metrics)
DecisionTreeClassifier_results['Model'] = 'DecisionTreeClassifier'
DecisionTreeClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
DecisionTreeClassifier_results = DecisionTreeClassifier_results[['Model',
                                                             'Sample',
                                                             'Mean CV F1 Score',
                                                             'Precision',
                                                             'Recall',
                                                             'F1 Score',
                                                             'Memory (MB) per read mapping',
                                                             'Execution Time (s) per read mapping',
                                                             'Memory (MB) 50k batch',
                                                             'Execution Time (s) 50k batch']]
DecisionTreeClassifier_results

In [ ]:
''' Aggregating AveragingMethods results '''
AveragingMethods_results = pd.concat([ExtraTreesClassifier_results,
                                      DecisionTreeClassifier_results])
AveragingMethods_shap_values = pd.concat([ExtraTreesClassifier_shap_values,
                                          DecisionTreeClassifier_shap_values])

## Naïve Bayes
Naïve Bayes algorithms represent a family of simple yet effective probabilistic classifiers based on applying Bayes' theorem with strong (naïve) independence assumptions between the features. Widely used in various applications, they are particularly known for their efficiency and ease of implementation.

#### Key Concepts

1. **Bayesian Theory**: The core principle of Naïve Bayes is Bayes' theorem, which describes the probability of a feature, based on prior knowledge of conditions that might be related to that feature.

2. **Feature Independence**: Naïve Bayes classifiers assume that the value of a particular feature is independent of the value of any other feature, given the class variable. This assumption simplifies the computation, hence the term "naïve."

3. **Variants of Naïve Bayes**:
   - **Gaussian Naïve Bayes**: Assumes that the continuous values associated with each class are distributed according to a Gaussian distribution.
   - **Multinomial Naïve Bayes**: Typically used for document classification, where the features are the frequencies of the words or tokens.
   - **Bernoulli Naïve Bayes**: Used in binary classification, especially text classification with 'bag of words' model.

4. **Applications**: Naïve Bayes classifiers are widely used in spam filtering, sentiment analysis, and document classification. They are also employed in medical diagnosis and weather prediction.

5. **Strengths and Limitations**:
   - **Strengths**: They are easy to implement, can handle both continuous and discrete data, and perform well in multi-class prediction. When the independence assumption holds, a Naïve Bayes classifier performs better compared to other models and requires much less training data.
   - **Limitations**: Their strong feature independence assumptions can lead to poor performance if this assumption does not hold. In practice, they are often outperformed by models like Random Forest or Gradient Boosting.

#### Practical Implementation

Implementing Naïve Bayes models involves careful preprocessing of data. For text data, techniques like bag-of-words or TF-IDF are common. Feature scaling is not required as the classifiers are not sensitive to the magnitude of data. Tuning involves choosing the right variant of Naïve Bayes and adjusting parameters like the smoothing factor in Multinomial and Bernoulli Naïve Bayes.

In summary, Naïve Bayes classifiers, with their basis in probability theory, offer a straightforward and efficient approach for building fast and scalable machine learning models. Their simplicity and the ability to make probabilistic predictions make them useful, especially in the initial stages of a modeling pipeline.


## Gaussian naive bayes classifier

The Gaussian Naive Bayes classifier is a probabilistic machine learning model particularly suited for continuous data, assuming that the features of each class are normally distributed.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import joblib
import time
import gc
import warnings
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Convert all values to numerical types, replacing non-numerical values with NaN
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    # Replace NaNs, infinities, and too large values
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=1000):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Accuracy': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Ensure data is in float64 format
    X_train = X_train.astype(np.float64)
    X_test = X_test.astype(np.float64)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'var_smoothing': np.logspace(0, -9, num=100)
    }

    model = GaussianNB()

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=1000)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')
    accuracy = accuracy_score(y_test, predicted)

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Accuracy'].append(accuracy)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained Gaussian Naive Bayes model
filename = 'model_weights/gaussian_nb_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.KernelExplainer(best_model.predict, X_train_resampled[:100])  # Use a subset of the training data
shap_values = explainer.shap_values(X_train_resampled[:100])

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()

In [ ]:
NaiveBayes_shap_values = pd.DataFrame(shap_values)
NaiveBayes_shap_values['Classifier'] = 'NaiveBayesClassifier'
NaiveBayes_shap_values

In [ ]:
NaiveBayes_results = pd.DataFrame(aggregate_metrics)
NaiveBayes_results['Model'] = 'NaiveBayesClassifier'
NaiveBayes_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
NaiveBayes_results = NaiveBayes_results[['Model',
                                                             'Sample',
                                                             'Mean CV F1 Score',
                                                             'Precision',
                                                             'Recall',
                                                             'F1 Score',
                                                             'Memory (MB) per read mapping',
                                                             'Execution Time (s) per read mapping',
                                                             'Memory (MB) 50k batch',
                                                             'Execution Time (s) 50k batch']]
NaiveBayes_results

In [ ]:
# """ GNB 2 """
# 
# import wandb
# import joblib
# import numpy as np
# import time
# from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# from memory_profiler import memory_usage
# import gc
# import pandas as pd
# 
# # Function to preprocess the data
# def preprocess_data(X_train, X_test):
#     # Convert all values to numerical types, replacing non-numerical values with NaN
#     X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce').values
#     X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce').values
# 
#     # Replace NaNs, infinities, and too large values
#     X_train = np.nan_to_num(X_train, nan=0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)
#     X_test = np.nan_to_num(X_test, nan=0.0, posinf=np.finfo(np.float64).max, neginf=np.finfo(np.float64).min)
# 
#     # Clip values to the range of float32 to avoid overflow
#     X_train = np.clip(X_train, np.finfo(np.float64).min, np.finfo(np.float64).max)
#     X_test = np.clip(X_test, np.finfo(np.float64).min, np.finfo(np.float64).max)
# 
#     return X_train, X_test
# 
# # Function to process data in batches
# def process_in_batches(model, X_test, batch_size=500):
#     predictions = []
#     total_memory = 0
#     total_time = 0
# 
#     for start in range(0, len(X_test), batch_size):
#         end = start + batch_size
#         batch = X_test[start:end]
# 
#         # Measure memory usage for this batch
#         mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
#         total_memory += max(mem_usage) - min(mem_usage)
# 
#         # Measure execution time for this batch
#         start_time = time.time()
#         preds = model.predict(batch)
#         total_time += time.time() - start_time
# 
#         predictions.extend(preds)
# 
#         # Explicitly clear memory
#         del batch
#         gc.collect()
# 
#     return np.array(predictions), total_memory, total_time
# 
# # Assuming datasets is a list of tuples with training and testing data
# datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
#             (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
#             (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
#             (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]
# 
# aggregate_metrics = {'Precision': [],
#                      'Recall': [],
#                      'F1 Score': [],
#                      'Accuracy': [],
#                      'Memory (MB) 50k batch': [],
#                      'Execution Time (s) 50k batch': [],
#                      'Memory (MB) per read mapping': [],
#                      'Execution Time (s) per read mapping': []}
# 
# for X_train, X_test, y_train, y_test in datasets:
#     # Preprocess the data
#     X_train, X_test = preprocess_data(X_train, X_test)
# 
#     # Standardize the data
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
# 
#     # Apply PCA for dimensionality reduction
#     pca = PCA(n_components=0.95)  # Retain 95% of variance
#     X_train = pca.fit_transform(X_train)
#     X_test = pca.transform(X_test)
# 
#     # Ensure data is in float64 format
#     X_train = X_train.astype(np.float64)
#     X_test = X_test.astype(np.float64)
# 
#     # Convert y_train and y_test to 1D array
#     y_train = y_train.squeeze()
#     y_test = y_test.squeeze()
# 
#     model = GaussianNB().fit(X_train, y_train)
# 
#     # Process in batches to handle memory usage
#     predicted, mem_usage, execution_time = process_in_batches(model, X_test, batch_size=500)
# 
#     # Metrics Calculation
#     precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')
#     accuracy = accuracy_score(y_test, predicted)
# 
#     execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
#     memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 
# 
#     # Aggregating metrics
#     aggregate_metrics['Precision'].append(precision)
#     aggregate_metrics['Recall'].append(recall)
#     aggregate_metrics['F1 Score'].append(fscore)
#     aggregate_metrics['Accuracy'].append(accuracy)
#     aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
#     aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
#     aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
#     aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)
# 
# # Save the last trained Gaussian Naive Bayes model
# filename = 'model_weights/gaussian_nb_model.joblib'
# joblib.dump(model, filename)
# 
# del model, X_test, X_train, y_train, y_test
# gc.collect()


## Gradient Boosting Algorithms
Gradient Boosting Algorithms are a group of powerful machine learning techniques that build predictive models in the form of an ensemble of weak prediction models, typically decision trees. They are known for their effectiveness in handling various types of data and their ability to improve the accuracy of predictions by reducing bias and variance.

#### Key Concepts

1. **Sequential Model Building**: Unlike other techniques that build models in parallel, gradient boosting builds one tree at a time, where each new tree helps to correct errors made by the previously trained tree.

2. **Loss Function Optimization**: These algorithms focus on minimizing a loss function iteratively. Each new model incrementally decreases the loss function of the entire system using the gradient descent method.

3. **Types of Gradient Boosting**:
   - **Gradient Boosting Machines (GBM)**: The traditional form of gradient boosting that sequentially adds predictors and corrects previous models.
   - **XGBoost (Extreme Gradient Boosting)**: An optimized distributed gradient boosting library designed to be highly efficient, flexible, and portable.
   - **LightGBM**: A gradient boosting framework that uses tree-based learning algorithms, designed for distributed and efficient training, particularly on large datasets.
   - **CatBoost**: An algorithm that can handle categorical data naturally and is robust to overfitting, making it particularly effective for a wide range of data science problems.

4. **Applications**: Gradient boosting models are used for a wide range of applications, including but not limited to ranking (like search engines), classification, regression, and many other machine learning tasks where high accuracy is desired.

5. **Strengths and Limitations**:
   - **Strengths**: They are highly accurate, can handle different types of data, and provide feature importance scores, which can be insightful for model interpretation.
   - **Limitations**: These models can be prone to overfitting if not tuned properly and are computationally more expensive than simpler models. They also require careful tuning of parameters and aren't as easy to interpret as simpler models.

#### Practical Implementation

Implementing gradient boosting models involves careful tuning of parameters like the number of trees, depth of trees, learning rate, and subsample ratio. The choice and tuning of the loss function are also crucial, depending on the specific problem. Due to their complexity, gradient boosting models often require more computational resources and time to train, especially on large datasets.

In summary, Gradient Boosting Algorithms are highly effective for complex machine learning problems where predictive accuracy is paramount. Their ability to iteratively correct errors and optimize performance makes them a go-to choice for competitive data science and a wide range of business applications.


### LGBM classifier
LightGBM (Light Gradient Boosting Machine) is an efficient and scalable implementation of gradient boosting that excels in handling large datasets and high-dimensional features, due to its novel approach of building trees leaf-wise rather than level-wise.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import joblib
import time
import gc
import warnings
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Convert all values to numerical types, replacing non-numerical values with NaN
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    # Replace NaNs, infinities, and too large values
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=500):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Accuracy': [],
                     'Memory (MB) 50k batch': [],
                     'Execution Time (s) 50k batch': [],
                     'Memory (MB) per read mapping': [],
                     'Execution Time (s) per read mapping': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=0.95)  # Retain 95% of variance
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Ensure data is in float32 format
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'num_leaves': [31, 50],
        'min_data_in_leaf': [20, 50, 100],
        'max_depth': [-1, 10, 20],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200]
    }

    model = LGBMClassifier()

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=500)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')
    accuracy = accuracy_score(y_test, predicted)

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Accuracy'].append(accuracy)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB) 50k batch'].append(mem_usage)
    aggregate_metrics['Execution Time (s) 50k batch'].append(execution_time)
    aggregate_metrics['Memory (MB) per read mapping'].append(memory_usage_per_read)
    aggregate_metrics['Execution Time (s) per read mapping'].append(execution_time_per_read)

# Save the last trained LightGBM model
filename = 'model_weights/lgbm_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()


In [ ]:
LGBMClassifier_shap_values = pd.DataFrame(shap_values)
LGBMClassifier_shap_values['Classifier'] = 'LGBMClassifier'
LGBMClassifier_shap_values

In [ ]:
LGBMClassifier_results = pd.DataFrame(aggregate_metrics)
LGBMClassifier_results['Model'] = 'LGBMClassifier'
LGBMClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
LGBMClassifier_results = LGBMClassifier_results[['Model',
                                                             'Sample',
                                                             'Mean CV F1 Score',
                                                             'Precision',
                                                             'Recall',
                                                             'F1 Score',
                                                             'Memory (MB) per read mapping',
                                                             'Execution Time (s) per read mapping',
                                                             'Memory (MB) 50k batch',
                                                             'Execution Time (s) 50k batch']]
LGBMClassifier_results

### XGB Classifier
The XGBoost (Extreme Gradient Boosting) Classifier is a highly efficient and scalable implementation of gradient boosting known for its performance and speed, often delivering state-of-the-art results in a wide range of machine learning tasks.

In [ ]:
import numpy as np
import pandas as pd
import time
import joblib
import warnings
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from imblearn.over_sampling import SMOTE
from memory_profiler import memory_usage
import gc
import shap

# Suppress specific warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Function to preprocess the data
def preprocess_data(X_train, X_test):
    # Convert all values to numerical types, replacing non-numerical values with NaN
    X_train = pd.DataFrame(X_train).apply(pd.to_numeric, errors='coerce')
    X_test = pd.DataFrame(X_test).apply(pd.to_numeric, errors='coerce')

    # Replace NaNs, infinities, and too large values
    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test

# Function to process data in batches
def process_in_batches(model, X_test, batch_size=500):
    predictions = []
    total_memory = 0
    total_time = 0

    for start in range(0, len(X_test), batch_size):
        end = start + batch_size
        batch = X_test[start:end]

        # Measure memory usage for this batch
        try:
            mem_usage = memory_usage((model.predict, (batch,)), interval=0.1)
            batch_mem_usage = max(mem_usage) - min(mem_usage)
            if batch_mem_usage == 0:
                batch_mem_usage = np.nan  # Ensure no zero values
        except Exception as e:
            batch_mem_usage = np.nan
        total_memory += batch_mem_usage

        # Measure execution time for this batch
        start_time = time.time()
        preds = model.predict(batch)
        total_time += time.time() - start_time

        predictions.extend(preds)

        # Explicitly clear memory
        del batch
        gc.collect()

    return np.array(predictions), total_memory, total_time

# Assuming datasets is a list of tuples with training and testing data
datasets = [(S1_X_train, S1_X_test, S1_y_train, S1_y_test),
            (S2_X_train, S2_X_test, S2_y_train, S2_y_test),
            (S3_X_train, S3_X_test, S3_y_train, S3_y_test),
            (S4_X_train, S4_X_test, S4_y_train, S4_y_test)]

aggregate_metrics = {'Precision': [],
                     'Recall': [],
                     'F1 Score': [],
                     'Accuracy': [],
                     'Memory (MB)': [],
                     'Execution Time (s)': [],
                     'Mean CV F1 Score': []}

# Create a placeholder for the best model
best_model = None

for X_train, X_test, y_train, y_test in datasets:
    # Preprocess the data
    X_train, X_test = preprocess_data(X_train, X_test)

    # Standardize the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Apply PCA for dimensionality reduction
    pca = PCA(n_components=0.95)  # Retain 95% of variance
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Ensure data is in float32 format
    X_train = X_train.astype(np.float32)
    X_test = X_test.astype(np.float32)

    # Convert y_train and y_test to 1D array
    y_train = y_train.squeeze()
    y_test = y_test.squeeze()

    # Handle class imbalance with SMOTE
    min_samples = min([sum(y_train == cls) for cls in np.unique(y_train)])
    if min_samples > 1:
        k_neighbors = min(5, min_samples - 1)
        smote = SMOTE(k_neighbors=k_neighbors)
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    else:
        # If a class has only one sample, duplicate it to allow splitting
        class_counts = pd.Series(y_train).value_counts()
        for cls in class_counts[class_counts == 1].index:
            X_train = np.vstack([X_train, X_train[y_train == cls]])
            y_train = np.hstack([y_train, y_train[y_train == cls]])
        X_train_resampled, y_train_resampled = X_train, y_train

    # Ensure the resampled data is in numpy array format
    X_train_resampled = np.array(X_train_resampled)
    y_train_resampled = np.array(y_train_resampled)

    # Cross-validation to detect overfitting
    sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

    # Adjust the test_size to avoid errors
    if len(np.unique(y_train_resampled)) > int(0.2 * len(y_train_resampled)):
        sss = StratifiedShuffleSplit(n_splits=5, test_size=len(np.unique(y_train_resampled)))

    # Hyperparameter tuning
    param_grid = {
        'max_depth': [3, 6, 10],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [100, 200, 500],
        'colsample_bytree': [0.3, 0.7]
    }

    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

    try:
        grid_search = GridSearchCV(model, param_grid, scoring='f1_weighted', cv=sss)
        grid_search.fit(X_train_resampled, y_train_resampled)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {e}")
        best_model = model

    cv_scores = []
    for train_index, test_index in sss.split(X_train_resampled, y_train_resampled):
        X_cv_train, X_cv_test = X_train_resampled[train_index], X_train_resampled[test_index]
        y_cv_train, y_cv_test = y_train_resampled[train_index], y_train_resampled[test_index]

        best_model.fit(X_cv_train, y_cv_train)
        y_cv_pred = best_model.predict(X_cv_test)
        _, _, f1, _ = precision_recall_fscore_support(y_cv_test, y_cv_pred, average='weighted')
        cv_scores.append(f1)

    print(f"Cross-validation F1 scores: {cv_scores}")
    print(f"Mean CV F1 score: {np.mean(cv_scores)}")

    best_model.fit(X_train_resampled, y_train_resampled)

    # Process in batches to handle memory usage
    predicted, mem_usage, execution_time = process_in_batches(best_model, X_test, batch_size=500)

    # Metrics Calculation
    precision, recall, fscore, support = precision_recall_fscore_support(y_test, predicted, average='weighted')
    accuracy = accuracy_score(y_test, predicted)

    execution_time_per_read = execution_time / len(X_test)  # per read mapping execution time 
    memory_usage_per_read = mem_usage / len(X_test)  # memory required per read mapping 

    # Aggregating metrics
    aggregate_metrics['Precision'].append(precision)
    aggregate_metrics['Recall'].append(recall)
    aggregate_metrics['F1 Score'].append(fscore)
    aggregate_metrics['Accuracy'].append(accuracy)
    aggregate_metrics['Mean CV F1 Score'].append(np.mean(cv_scores))
    aggregate_metrics['Memory (MB)'].append(mem_usage)
    aggregate_metrics['Execution Time (s)'].append(execution_time)

# Final Aggregation
for metric, values in aggregate_metrics.items():
    print({f'Aggregate {metric}': np.mean(values)})

# Save the last trained XGBoost model
filename = 'model_weights/xgboost_model.joblib'
joblib.dump(best_model, filename)

# SHAP Explainer
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_train_resampled[:100])  # Use a subset of the training data

# SHAP summary plot with feature names
feature_names = [f"Feature {i}" for i in range(X_train_resampled.shape[1])]
shap.summary_plot(shap_values, X_train_resampled[:100], feature_names=feature_names)

del best_model, X_test, X_train, y_train, y_test
gc.collect()


In [ ]:
XGBClassifier_shap_values = pd.DataFrame(shap_values)
XGBClassifier_shap_values['Classifier'] = 'XGBoostClassifier'
XGBClassifier_shap_values

In [ ]:
XGBClassifier_results = pd.DataFrame(aggregate_metrics)
XGBClassifier_results['Model'] = 'XGBoostClassifier'
XGBClassifier_results['Sample'] = ['S1', 'S2', 'S3', 'S4']
XGBClassifier_results = XGBClassifier_results[['Model',
                                                             'Sample',
                                                             'Mean CV F1 Score',
                                                             'Precision',
                                                             'Recall',
                                                             'F1 Score',
                                                             'Memory (MB) per read mapping',
                                                             'Execution Time (s) per read mapping',
                                                             'Memory (MB) 50k batch',
                                                             'Execution Time (s) 50k batch']]
XGBClassifier_results

In [ ]:
''' Aggregating Gradient Boosting results '''
GradientBoosting_results = pd.concat([XGBClassifier_results,
                                      LGBMClassifier_results
                                      ])
GradientBoosting_shap_values = pd.concat([XGBClassifier_shap_values,
                                          LGBMClassifier_shap_values
                                          ])